In [1]:
import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizer, DistilBertModel
import os
import time
from web_scrapping import *
from functions import *

In [2]:
df_books = pd.read_csv("data/goodreads_data.csv")

In [3]:
df_books

,Unnamed: 0,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL
0,0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...
1,1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...
2,2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...
3,3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...
4,4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...
...,...,...,...,...,...,...,...,...
9995,9995,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,How far would you go? If human society was gen...,"['Dystopia', 'Science Fiction', 'Post Apocalyp...",3.44,276,https://www.goodreads.com/book/show/22085400-b...
9996,9996,Dynamo,Eleanor Gustafson,Jeth Cavanaugh is searching for a new life alo...,[],4.23,60,https://www.goodreads.com/book/show/20862902-d...
9997,9997,The Republic of Trees,Sam Taylor,This dark fable tells the story of four Englis...,"['Fiction', 'Horror', 'Dystopia', 'Coming Of A...",3.29,383,https://www.goodreads.com/book/show/891262.The...
9998,9998,"Waking Up (Healing Hearts, #1)",Renee Dyer,For Adriana Monroe life couldn’t get any bette...,"['New Adult', 'Romance', 'Contemporary Romance...",4.13,263,https://www.goodreads.com/book/show/19347252-w...


In [4]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  9923 non-null   object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  object 
 7   URL          10000 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 625.1+ KB


In [5]:
df_books["Description"][0]

'The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.'

In [3]:
numerico(df_books, "Num_Ratings", "Description")

In [4]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   10000 non-null  int64  
 1   Book         10000 non-null  object 
 2   Author       10000 non-null  object 
 3   Description  10000 non-null  object 
 4   Genres       10000 non-null  object 
 5   Avg_Rating   10000 non-null  float64
 6   Num_Ratings  10000 non-null  int32  
 7   URL          10000 non-null  object 
dtypes: float64(1), int32(1), int64(1), object(5)
memory usage: 586.1+ KB


Crear una nueva columna content que combine el título y la descripción, tal como lo has hecho. Además, vamos a eliminar cualquier columna innecesaria (como Unnamed: 0)

In [4]:
# Crear una columna 'content' combinando 'Book' y 'Description'
df_books['content'] = df_books['Book'] + " " + df_books['Description'].fillna("")

# Eliminar columna innecesaria
df_books = df_books.drop(columns=['Unnamed: 0'])

In [6]:
df_books.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Book         10000 non-null  object 
 1   Author       10000 non-null  object 
 2   Description  10000 non-null  object 
 3   Genres       10000 non-null  object 
 4   Avg_Rating   10000 non-null  float64
 5   Num_Ratings  10000 non-null  int32  
 6   URL          10000 non-null  object 
 7   content      10000 non-null  object 
dtypes: float64(1), int32(1), object(6)
memory usage: 586.1+ KB


In [7]:
df_books

,Book,Author,Description,Genres,Avg_Rating,Num_Ratings,URL,content
0,To Kill a Mockingbird,Harper Lee,The unforgettable novel of a childhood in a sl...,"['Classics', 'Fiction', 'Historical Fiction', ...",4.27,"5,691,311",https://www.goodreads.com/book/show/2657.To_Ki...,To Kill a Mockingbird The unforgettable novel ...
1,Harry Potter and the Philosopher’s Stone (Harr...,J.K. Rowling,Harry Potter thinks he is an ordinary boy - un...,"['Fantasy', 'Fiction', 'Young Adult', 'Magic',...",4.47,"9,278,135",https://www.goodreads.com/book/show/72193.Harr...,Harry Potter and the Philosopher’s Stone (Harr...
2,Pride and Prejudice,Jane Austen,"Since its immediate success in 1813, Pride and...","['Classics', 'Fiction', 'Romance', 'Historical...",4.28,"3,944,155",https://www.goodreads.com/book/show/1885.Pride...,Pride and Prejudice Since its immediate succes...
3,The Diary of a Young Girl,Anne Frank,Discovered in the attic in which she spent the...,"['Classics', 'Nonfiction', 'History', 'Biograp...",4.18,"3,488,438",https://www.goodreads.com/book/show/48855.The_...,The Diary of a Young Girl Discovered in the at...
4,Animal Farm,George Orwell,Librarian's note: There is an Alternate Cover ...,"['Classics', 'Fiction', 'Dystopia', 'Fantasy',...",3.98,"3,575,172",https://www.goodreads.com/book/show/170448.Ani...,Animal Farm Librarian's note: There is an Alte...
...,...,...,...,...,...,...,...,...
9995,"Breeders (Breeders Trilogy, #1)",Ashley Quigley,How far would you go? If human society was gen...,"['Dystopia', 'Science Fiction', 'Post Apocalyp...",3.44,276,https://www.goodreads.com/book/show/22085400-b...,"Breeders (Breeders Trilogy, #1) How far would ..."
9996,Dynamo,Eleanor Gustafson,Jeth Cavanaugh is searching for a new life alo...,[],4.23,60,https://www.goodreads.com/book/show/20862902-d...,Dynamo Jeth Cavanaugh is searching for a new l...
9997,The Republic of Trees,Sam Taylor,This dark fable tells the story of four Englis...,"['Fiction', 'Horror', 'Dystopia', 'Coming Of A...",3.29,383,https://www.goodreads.com/book/show/891262.The...,The Republic of Trees This dark fable tells th...
9998,"Waking Up (Healing Hearts, #1)",Renee Dyer,For Adriana Monroe life couldn’t get any bette...,"['New Adult', 'Romance', 'Contemporary Romance...",4.13,263,https://www.goodreads.com/book/show/19347252-w...,"Waking Up (Healing Hearts, #1) For Adriana Mon..."


In [10]:
df_books["content"][0]

'To Kill a Mockingbird The unforgettable novel of a childhood in a sleepy Southern town and the crisis of conscience that rocked it. "To Kill A Mockingbird" became both an instant bestseller and a critical success when it was first published in 1960. It went on to win the Pulitzer Prize in 1961 and was later made into an Academy Award-winning film, also a classic.Compassionate, dramatic, and deeply moving, "To Kill A Mockingbird" takes readers to the roots of human behavior - to innocence and experience, kindness and cruelty, love and hatred, humor and pathos. Now with over 18 million copies in print and translated into forty languages, this regional story by a young Alabama woman claims universal appeal. Harper Lee always considered her book to be a simple love story. Today it is regarded as a masterpiece of American literature.'

Guardar csv

In [5]:
df_books.to_csv("data/df_books.csv", index=False)

Web scrapping busqueda imagenes

In [7]:
# Extraer imágenes
start_time = time.time()  # Tiempo de inicio

for index, row in df_books.iterrows():
    df_books.at[index, 'Image_URL'] = get_cover_image_url(row['URL'])
    print(f"Imagen {index + 1}/{len(df_books)} adquirida")  # Mensaje de progreso
    
    # Guardar el DataFrame después de cada imagen
    df_books.to_csv("data/goodreads_data_with_images.csv", index=False)
    print(f"Datos guardados después de la imagen {index + 1}")

total_time = time.time() - start_time  # Tiempo total de ejecución
print(f"Tiempo total de ejecución: {total_time:.2f} segundos")  # Mensaje final

Procesando URL: https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird
Imagen encontrada para: https://www.goodreads.com/book/show/2657.To_Kill_a_Mockingbird
Imagen 1/10000 adquirida
Datos guardados después de la imagen 1
Procesando URL: https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone
Imagen encontrada para: https://www.goodreads.com/book/show/72193.Harry_Potter_and_the_Philosopher_s_Stone
Imagen 2/10000 adquirida
Datos guardados después de la imagen 2
Procesando URL: https://www.goodreads.com/book/show/1885.Pride_and_Prejudice
Imagen encontrada para: https://www.goodreads.com/book/show/1885.Pride_and_Prejudice
Imagen 3/10000 adquirida
Datos guardados después de la imagen 3
Procesando URL: https://www.goodreads.com/book/show/48855.The_Diary_of_a_Young_Girl
Imagen encontrada para: https://www.goodreads.com/book/show/48855.The_Diary_of_a_Young_Girl
Imagen 4/10000 adquirida
Datos guardados después de la imagen 4
Procesando URL: https://www.goodre